In [5]:
import dotenv
dotenv.load_dotenv()

True

# Conversational bot for Madkudu services

The whole idea is to use 

# 1. Load data from Madkudu



In [6]:

from langchain.indexes import VectorstoreIndexCreator

I could have also chosen to download some of these pdf files directly https://support.madkudu.com/hc/en-us/article_attachments/360052888911 Instead of making a crawler

In [28]:

from langchain.document_loaders import TextLoader
loader = TextLoader('all.txt', encoding='utf-8')
documents = loader.load()

# 2. Split data into chunks

RecursiveCharacterTextSplitter is more optimized than the CharacterTextSplitter

In [48]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
#Alternative option
#text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)

text_splitter = RecursiveCharacterTextSplitter(
    #i chose 500 because the data is not that 
    # number of characters
    chunk_size = 500, 
    #How many chars are going to be overlapped in between each chunk
    chunk_overlap = 250,
    
    length_function = len,
    
    #includes index of character where the separation between each chunk is done
    add_start_index = True
    
    )

chunks = text_splitter.split_documents(documents)

The data was split into 113 chunks of 500 chars 

In [49]:
len(chunks)

113

In [50]:
chunks[15].page_content

"Static enrichment: MadKudu partners with Clearbit, HG data, and PredictLeads to enrich your data. Are you already purchasing through ZoomInfo, Bombora, Datafox? No problem, MadKudu can also use this data. \nCRM information: actually any data from your CRM (Leads, Contacts, Accounts, Opportunities), be it provided by a 3rd party or from your Marketing and Sales input, can be pulled to MadKudu's platform."

Source could have been useful if I separated each url into different txt files to have a better documentation. So perhaps I can implement this at the end.

In [51]:
chunks[15].metadata

{'source': 'all.txt', 'start_index': 3951}

# 3. Store chunks in a database - Chroma

I will need to use my OpenAi account to access to chroma and to use the embedding tool to embed each chunk.

We are going to create and save to disk a persistent directory in case I want to scale this in the future. This way I can copy the directory and deploy it aa a file. Example: accesing to this database through a Lambda function

In [60]:
from langchain.document_loaders import DirectoryLoader
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
import os
import shutil

CHROMA_PATH = "chroma"
DATA_PATH = "data/books"

In [61]:

def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

In [56]:
! pip install chromadb

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/525.5 kB ? eta -:--:--
     -------------- ----------------------- 204.8/525.5 kB 4.1 MB/s eta 0:00:01
     -------------------------------- ----- 450.6/525.5 kB 5.6 MB/s eta 0:00:01
     -------------------------------------- 525.5/525.5 kB 4.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/151.6 kB ? eta -:--:--
     -------------------------------------- 151.6/151.6 kB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/92.1 kB ? eta -:--:--
     ---------------------------------------- 92.1/92.1 kB ? eta 0:00:00
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/41.3 kB ? eta -:--:--
     ---------------------------------------- 41.3/41.3 kB 1.9 MB/s eta 0:00:00
     --

ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'C:\\Users\\matts\\AppData\\Roaming\\Python\\Python311\\site-packages\\~-mpy\\.libs\\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll'
Check the permissions.


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\matts\AppData\Roaming\Python\Python311\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=5>
  return process_handler(cmd, _system_body)
C:\Users\matts\AppData\Roaming\Python\Python311\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)
C:\Users\matts\AppData\Roaming\Python\Python311\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=7>
  return process_handler(cmd, _system_body)
sys:1: R

In [62]:
save_to_chroma(chunks)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [33]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

Embed and store the chunks to facilitate the search accross them